In [1]:
# This version uses a model architecture with two LSTM layers and a dropout layer in between
# again, just using part1 data for training and validation

# Here I will be building out the architecture of the first classification LSTM
# At each time step, this LSTM will take in a vector representing the extracted audio and visual features from Liris-ACCEDE
# Its goal is to output whether or not the movie induces fear at each time step


# First, import necessary libraries
import tensorflow as tf
import numpy as np

/Users/chloeloughridge/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:
# setting up the keras stuff
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers import LSTM
# my custom data_utils file
from data_utils_local08 import *

Using TensorFlow backend.


In [3]:
# uploading the X values
X_input = load_Xinput(get_fc6_directory(7))
print(X_input.shape)

(212, 4096)


In [4]:
# uploading the Y values
y_data_input = fear_oneHot(212, 'fear_annotations_part01/MEDIAEVAL18_7_Fear.txt')
print(y_data_input.shape)

(212,)


In [5]:
# Uploading part01 of the local fc6 data

timesteps = 212   # the number of seconds in movie 07 --> i will figure out how to programmatically get this value
data_dim = 4096    # the number of output values from VGG16 layer fc6 --> switch to programmatic later
num_movies = 4
batch_size = 7
num_epochs = 5
validation_num = 3

# set up the X_train_data master array
X_train_data = np.zeros([num_movies, timesteps, data_dim]) #oooooh this array will have to be as long as the longest
                                                            # movie and padded with zeros --> this won't cause problems
                                                            # right?
X_valid_data = np.zeros([validation_num, timesteps, data_dim])
        
# for each movie number between and including 7 and 13
for num in range(num_movies):
    # load the X_input data
    X_input = load_Xinput(get_fc6_directory(7 + num))
    # put this X_input data into the X_train_data array
    X_train_data[num, :, :] = X_input
print(X_train_data.shape)



(4, 212, 4096)


In [6]:
# loading X validation set
X_valid_data = np.zeros([validation_num, timesteps, data_dim])

for num in range(validation_num):
    # load the X_input data
    X_valid = load_Xinput(get_fc6_directory(7 + num_movies + num))
    # put this X_input data into the X_train_data array
    X_valid_data[num, :, :] = X_valid
print(X_valid_data.shape)


(3, 212, 4096)


In [7]:
# uploading the y data

# set up y_train_data master array
Y_train_data = np.zeros([num_movies, timesteps])

# for each movie number between and including 7 and 13
for num in range(num_movies):
    # create the appropriate path to the fear annotation data
    #print(num)
    path = os.path.join('fear_annotations_part01/MEDIAEVAL18_{}_Fear.txt'.format(7+num))
    # create a one-hot vector
    y_data = fear_oneHot(timesteps, path)
    # add this one-hot vector to y_train_data
    Y_train_data[num, :] = y_data
print(Y_train_data[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [8]:
# upload the Y validation set
Y_valid_data = np.zeros([validation_num, timesteps])

# for each movie number in validation set
for num in range(validation_num):
    # create the appropriate path to the fear annotation data
    #print(num)
    path = os.path.join('fear_annotations_part01/MEDIAEVAL18_{}_Fear.txt'.format(7+ num_movies + num))
    # create a one-hot vector
    y_valid = fear_oneHot(timesteps, path)
    # add this one-hot vector to y_train_data
    Y_valid_data[num, :] = y_valid
print(Y_valid_data[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [9]:
# model architecture
model = Sequential()
model.add(LSTM(212, return_sequences=True,
               input_shape=(timesteps, data_dim)))
# dropout layer
model.add(Dropout(0.5))

# another LSTM layer
model.add(LSTM(212, return_sequences=True))

# necessary flatten layer
model.add(Flatten()) 

# add the final dense layer and then softmax
model.add(Dense(212, activation='softmax'))

In [10]:
# compiling LSTM model
# note that Ng used an Adam optimizer and categorical cross-entropy loss
# but this is a binary classification problem so I think the parameters below should suffice
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['binary_accuracy'])

In [11]:
# running the LSTM model
model.fit(X_train_data, Y_train_data, epochs = 20, validation_data=(X_valid_data, Y_valid_data))
print("finished training!")

Train on 4 samples, validate on 3 samples
Epoch 1/20
4/4 [==============================] - 4s 1s/step - loss: 1.0507 - binary_accuracy: 0.8054 - val_loss: 2.0607 - val_binary_accuracy: 0.7689
Epoch 2/20
4/4 [==============================] - 3s 762ms/step - loss: 0.9899 - binary_accuracy: 0.8054 - val_loss: 2.5496 - val_binary_accuracy: 0.7673
Epoch 3/20
4/4 [==============================] - 3s 750ms/step - loss: 2.5300 - binary_accuracy: 0.8031 - val_loss: 3.0866 - val_binary_accuracy: 0.7689
Epoch 4/20
4/4 [==============================] - 3s 794ms/step - loss: 2.2222 - binary_accuracy: 0.8054 - val_loss: 3.2074 - val_binary_accuracy: 0.7689
Epoch 5/20
4/4 [==============================] - 3s 851ms/step - loss: 2.0226 - binary_accuracy: 0.8066 - val_loss: 3.4491 - val_binary_accuracy: 0.7689
Epoch 6/20
4/4 [==============================] - 3s 726ms/step - loss: 1.9936 - binary_accuracy: 0.8054 - val_loss: 3.5082 - val_binary_accuracy: 0.7689
Epoch 7/20
4/4 [=====================

In [16]:
# trying to view the model output
out = model.predict(X_train_data)
print("model prediction:")
print(out[0])
print("target:")
print(Y_train_data[0])

print("before 64:")
print(out[0][63])
print("64:")
print(out[0][65])

print("rounded")
print(np.round(out)[0])

model prediction:
[5.80215828e-06 4.54788733e-06 7.53379300e-06 5.51097401e-06
 8.46636067e-06 4.16587000e-06 4.93033122e-06 7.82797997e-06
 5.10781183e-06 5.04128957e-06 4.56491625e-06 9.48049001e-06
 6.48030573e-06 5.32008380e-06 1.15500889e-05 8.18212447e-06
 6.61045624e-06 6.81488382e-06 7.07102527e-06 8.40262237e-06
 1.21017738e-05 5.32808008e-06 7.58328542e-06 7.86852434e-06
 6.26041174e-06 4.04386219e-06 2.05821616e-05 6.65964080e-06
 2.86260856e-05 1.84637629e-05 1.08862459e-05 2.60548477e-05
 9.00484611e-06 8.30485624e-06 1.16883539e-05 1.18694952e-05
 1.13861606e-05 5.97758801e-04 8.00490438e-04 9.67328378e-06
 1.75866007e-05 2.22817471e-05 9.39848087e-06 1.50678961e-05
 1.63513778e-05 9.91435627e-06 1.13243377e-05 3.24915381e-05
 9.77735453e-06 2.03323489e-05 1.75254736e-05 1.30615981e-05
 8.09948833e-04 6.45566499e-04 7.72805826e-04 6.33836025e-04
 7.64632598e-04 8.44476977e-04 8.21812137e-04 8.42015841e-04
 1.12248808e-05 1.16809661e-05 1.71042466e-05 9.41981943e-06
 9.987

In [ ]:
#try visualizing this model at some point?